In [ ]:
%%time
import pandas as pd

def TransformColumns(Df):
    Df.columns = [c.lower().replace(' ', '_') for c in Df.columns]
    return None


dictMonths = {1: 'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun', 7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'}



varMonth = input("Enter the first MONTH of the forecast that should be reviewed: ")
varYear = input("Enter the YEAR of the first month that should be reviewed: ")

varQuery = 'ano == '+str(varYear)+' or (ano == '+str(int(varYear) - 1)+' and mes >= '+str(varMonth)+')'



dfProd = pd.read_excel('./Inputs/1_PRODUTO ACABADO.xls', sheet_name='Plan1', usecols='A:D,P,Q,EB')
dfCatAnalyst = pd.read_excel('./Inputs/0_Cat por analista.xls')

TransformColumns(dfProd)
dfProd = dfProd.loc[dfProd.linha != 'FORA DE LINHA', :]
dfProd = dfProd.loc[dfProd.linha != 'INDISPONIVEL SEM RECOMPRA', :]
dfProd = dfProd.loc[dfProd.griffe == 'IMAGINARIUM', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'ACESSORIOS', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'INSUMOS', :]

dfProd = dfProd.merge(dfCatAnalyst, on='grupo_produto', how='left')



dfStockStores = pd.read_excel('./Inputs/1_ESTOQUE REDE.xls', sheet_name='Plan1')
dfECMandDC = pd.read_excel('./Inputs/1_ESTOQUE CD E ECM.xls', sheet_name='Plan1')

TransformColumns(dfStockStores)
TransformColumns(dfECMandDC)

dfECMandDC['estoque_cd'] = dfECMandDC['disponivel_venda'] + dfECMandDC['movimentação_cd']

dfECM = dfECMandDC.loc[dfECMandDC.filial == 'ECOMMERCE', :]
dfDC = dfECMandDC.loc[dfECMandDC.filial == 'IMAGINARIUM SAO JOSE', :]



dfVendas = pd.read_excel('./Inputs/2_VENDAS.xlsx', usecols='A, D, L, N, O')
TransformColumns(dfVendas)
dfVendasLTM = dfVendas.query(varQuery)
pivotVendasLTM = dfVendasLTM.copy(deep=True)
pivotVendasLTM['ano_e_mes'] = pivotVendasLTM.apply(lambda x: str(dictMeses[x['mes']])+'/'+str(x['ano']), axis=1)
pivotVendasLTM = pivotVendasLTM.pivot_table(values='venda', index='produto', columns='ano_e_mes', aggfunc='sum')

mesmin = dfVendasLTM['data_registro'].min().month
anomin = dfVendasLTM['data_registro'].min().year

dfDFU = dfProd.copy(deep=True)


for m in range(0, 12):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin
    else:
        mes = mesmin + m -12
        ano = anomin +1

    mes_ano = str(dictMeses[mes])+'/'+str(ano)
    dfDFU = dfDFU.merge(pivotVendasLTM[[mes_ano]], left_on='produto', right_index=True, how='left')



dfDFU = dfDFU.merge(dfStockStores[['produto', 'estoque']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque": "rede"}, inplace=True)

dfDFU = dfDFU.merge(dfDC[['produto', 'estoque_cd']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque_cd": "cd"}, inplace=True)

dfDFU = dfDFU.merge(dfECM[['produto', 'estoque_cd']], on='produto', how='left')
dfDFU.rename(index=str, columns={"estoque_cd": "ecm"}, inplace=True)

dfDFU = dfDFU.fillna(0)
dfDFU['estoque_rede+cd'] = dfDFU['rede'] + dfDFU['cd']



dfDFUAntiga = pd.read_excel('./Inputs/2_DFU ANTIGA.xlsx')

mesesFuturos =[]

for m in range(0, 14):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin + 1
    else:
        mes = mesmin + m -12
        ano = anomin + 2
    mes_ano = str(dictMeses[mes])+'/'+str(ano)
    mesesFuturos.append(mes_ano)


for m in mesesFuturos[:-1]:
    dfDFU[m] = dfDFU.apply(lambda x: dfDFUAntiga.loc[(dfDFUAntiga['produto'] == x['produto'])][m].sum(), axis=1)
for m in mesesFuturos[-1:]:
    dfDFU[m] = dfDFU[dfDFU.columns.tolist()[-1]]




dfDFU.to_excel("DFUNova.xlsx", sheet_name='Base', index=False)